# Creating and manipulating DB

### Creating Tables with SQLAlchemy

In [1]:
from sqlalchemy import Table, Column, String, Integer, Float, Boolean
from sqlalchemy import create_engine, MetaData

In [2]:
engine = create_engine('sqlite:///db_create.sqlite')
metadata = MetaData()
connection = engine.connect()

In [3]:
data = Table('data',metadata,
             Column('name', String(255)),
             Column('count', Integer()),
             Column('amount', Float()),
             Column('valid', Boolean())
            )

In [4]:
metadata.create_all(engine)
print(repr(data))

Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>), schema=None)


### Constraints and Data Defaults

In [5]:
data = Table('data', metadata,
             Column('name', String(255), unique=True),
             Column('count', Integer(), default=1),
             Column('amount', Float()),
             Column('valid', Boolean(), default=False),
             extend_existing=True
)

In [6]:
metadata.create_all(engine)
repr(data)

"Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>, default=ColumnDefault(1)), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>, default=ColumnDefault(False)), schema=None)"

### Inserting a single row with an insert() statement

In [7]:
from sqlalchemy import insert, select

In [8]:
stmt = insert(data).values(
    name='Anna',
    count=1,
    amount=1000.00,
    valid=True
)

In [9]:
results = connection.execute(stmt)

In [10]:
results.rowcount

1

In [11]:
stmt = select([data]).where(data.columns.name=='Anna')
connection.execute(stmt).first()

('Anna', 1, 1000.0, True)

### Inserting Multiple Records at Once

In [12]:
values_list = [
    {'name': 'Anna', 'count': 1, 'amount': 900.00, 'valid': True},
    {'name': 'Taylor', 'count': 1, 'amount': 750.00, 'valid': False}
]

In [13]:
stmt = insert(data)
results = connection.execute(stmt, values_list)
results.rowcount

2

In [14]:
stmt = select([data])
connection.execute(stmt).fetchall()

[('Anna', 1, 1000.0, True),
 ('Anna', 1, 900.0, True),
 ('Taylor', 1, 750.0, False),
 ('Anna', 1, 1000.0, True),
 ('Anna', 1, 900.0, True),
 ('Taylor', 1, 750.0, False),
 ('Anna', 1, 1000.0, True),
 ('Anna', 1, 900.0, True),
 ('Taylor', 1, 750.0, False)]

### Loading a CSV into a Table

In [15]:
engine = create_engine('sqlite:///census_csv.sqlite')
metadata = MetaData()
connection = engine.connect()

In [16]:
census = Table('census',metadata,
             Column('state', String(255)),
             Column('sex', String(255)),
             Column('age', Integer()),
             Column('pop2000', Float()),
             Column('pop2008', Float())
            )
metadata.create_all(engine)
repr(census)

"Table('census', MetaData(bind=None), Column('state', String(length=255), table=<census>), Column('sex', String(length=255), table=<census>), Column('age', Integer(), table=<census>), Column('pop2000', Float(), table=<census>), Column('pop2008', Float(), table=<census>), schema=None)"

In [17]:
import csv
stmt = insert(census)
values_list = []
total_rowcount = 0

In [18]:
with open('census.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')
    for idx, row in enumerate(csv_reader):
        data = {
            'state': row[0],
            'sex': row[1],
            'age': row[2],
            'pop2000': row[3],
            'pop2008': row[4]
        }
        values_list.append(data)
        
        if idx % 51 == 0:
            results = connection.execute(stmt, values_list)
            total_rowcount += results.rowcount
            values_list = []

In [19]:
total_rowcount

8722

### Updating individual records

In [20]:
from sqlalchemy import update

In [21]:
engine = create_engine('sqlite:///census.sqlite')
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)
state_fact = Table('state_fact', metadata, autoload=True, autoload_with=engine)
connection = engine.connect()

In [22]:
engine.table_names()

['census', 'state_fact']

In [23]:
select_stmt = select([state_fact]).where(state_fact.columns.name == 'New York')
connection.execute(select_stmt).fetchall()

[('32', 'New York', 'NY', 'USA', 'state', '10', 'current', 'occupied', '', '36', 'N.Y.', 'II', '1', 'Northeast', '2', 'Mid-Atlantic', '2')]

In [24]:
stmt = update(state_fact).values(fips_state = 36)

In [25]:
stmt = stmt.where(state_fact.columns.name == 'New York')
results = connection.execute(stmt)
results.rowcount

1

In [26]:
connection.execute(select_stmt).fetchall()

[('32', 'New York', 'NY', 'USA', 'state', '10', 'current', 'occupied', '', '36', 'N.Y.', 'II', '1', 'Northeast', '2', 'Mid-Atlantic', '2')]

### Updating Multiple Records

In [27]:
stmt = update(state_fact).values(notes='The Wild West')
stmt = stmt.where(state_fact.columns.census_region_name == 'West')

In [28]:
results = connection.execute(stmt)
results.rowcount

13

### Correlated Updates

In [29]:
flat_census = Table('flat_census',metadata,
             Column('state_name', String(255)),
             Column('fips_code', Integer())
            )
metadata.create_all(engine)
repr(flat_census)

"Table('flat_census', MetaData(bind=None), Column('state_name', String(length=255), table=<flat_census>), Column('fips_code', Integer(), table=<flat_census>), schema=None)"

In [30]:
engine.table_names()

['census', 'flat_census', 'state_fact']

In [31]:
flat_census_list = [('Illinois', '17'),
 ('New Jersey', '34'),
 ('North Dakota', '38'),
 ('Oregon', '41'),
 ('Washington DC', '11'),
 ('Wisconsin', '55'),
 ('Arizona', '4'),
 ('Arkansas', '5'),
 ('Colorado', '8'),
 ('Hawaii', '15'),
 ('Kansas', '20'),
 ('Louisiana', '22'),
 ('Montana', '30'),
 ('Nebraska', '31'),
 ('Oklahoma', '40'),
 ('Idaho', '16'),
 ('Massachusetts', '25'),
 ('Michigan', '26'),
 ('Missouri', '29'),
 ('North Carolina', '37'),
 ('Ohio', '39'),
 ('Rhode Island', '44'),
 ('South Carolina', '45'),
 ('Wyoming', '56'),
 ('Indiana', '18'),
 ('Pennsylvania', '42'),
 ('South Dakota', '46'),
 ('Tennessee', '47'),
 ('Vermont', '50'),
 ('Alaska', '2'),
 ('Delaware', '10'),
 ('Kentucky', '21'),
 ('Mississippi', '28'),
 ('Virginia', '51'),
 ('Florida', '12'),
 ('Maryland', '24'),
 ('Nevada', '32'),
 ('Washington', '53'),
 ('California', '6'),
 ('Connecticut', '9'),
 ('Georgia', '13'),
 ('Iowa', '19'),
 ('Maine', '23'),
 ('New Hampshire', '33'),
 ('New Mexico', '35'),
 ('Texas', '48'),
 ('Alabama', '1'),
 ('Minnesota', '27'),
 ('New York', '36'),
 ('Utah', '49'),
 ('West Virginia', '54')]

In [32]:
stmt = insert(flat_census)
values_list = []
for row in flat_census_list:
    data = {
        'state_name': row[0],
        'fips_code': row[1]
    }
    values_list.append(data)

results = connection.execute(stmt, values_list)
results.rowcount

51

In [33]:
fips_stmt = select([state_fact.columns.name])

In [34]:
fips_stmt = fips_stmt.where(
    state_fact.columns.fips_state == flat_census.columns.fips_code)

In [35]:
update_stmt = update(flat_census).values(state_name=fips_stmt)
results = connection.execute(update_stmt)
results.rowcount

51

### Deleting all the records from a table

In [36]:
from sqlalchemy import delete, func, and_

In [37]:
# stmt = delete(census)
# results = connection.execute(stmt)

### Deleting specific records

In [38]:
stmt = select([func.count(census.columns.sex)]).where(and_(
            census.columns.sex=='M',
            census.columns.age==36)
            )

In [39]:
connection.execute(stmt).scalar()

51

In [40]:
stmt_del = delete(census)
stmt_del = stmt_del.where(
            and_(
                census.columns.sex == 'M',
                census.columns.age == 36))
results = connection.execute(stmt_del)
results.rowcount

51

### Deleting a Table Completely

In [41]:
# state_fact.drop(engine)
# print(state_fact.exists(engine))
# metadata.drop_all(engine)
# print(census.exists(engine))